In [1]:
import pandas as pd
import numpy as np
import glob

In [2]:
def prepare_collapsed_files(file, level):
    
    # import data
    df = pd.read_csv(file, sep="\t", header=1)

    # set index
    df = df.set_index('#OTU ID')
    df.index.name = ""
    
    # transpose
    df = df.T

    # reset index
    df = df.reset_index()

    # rename column
    df = df.rename(columns = {'index':'SampleID'})

    # melt
    df = pd.melt(df, id_vars =['SampleID'])

    # rename
    df = df.rename(columns = {'':'Taxa', 'value':'pAbundance'})

    # replace characters in Taxa
    df['Taxa'] = df['Taxa'].replace(['k__','p__','c__','o__','f__','g__','s__','__'], ["","","","","","","",""], regex=True)

    # split each taxon
    split_taxa = df['Taxa'].str.split(';', expand=True)
    split_taxa.columns = ["level"+str(x+1) for x in split_taxa.columns]

    # get specific level
    level_taxa = pd.DataFrame(split_taxa.loc[:,level])

    # replace blanks with unassigned
    df = pd.concat([df, level_taxa], axis=1).replace("", "Unassigned")

    # groupby SampleID and specific level
    df = df.groupby(['SampleID',level]).agg('sum')

    # reset index
    df = df.reset_index()

    # pivot for normalization
    df = df.pivot(index='SampleID', columns=level, values='pAbundance')

    # normalization
    df = df.div(df.sum(axis=1), axis=0)

    # reset index again because pivot indexes SampleID
    df = df.reset_index()

    # melt for ggplot
    df = pd.melt(df, id_vars =['SampleID'])

    # rename column
    df = df.rename(columns = {level:'Taxa', 'value':'pAbundance'})

    # merge metadata
    df = df.merge(metadata, on='SampleID')
    
    # drop rows that are zero
    df = df.loc[df['pAbundance']>0,:]
    
    return df

In [27]:
df = pd.read_csv('pmi_taxonomy_control_samples.csv')
s = df.iloc[:, 1:120]
s_list = s.iloc[:,:-1].columns.to_list()
s = pd.melt(s, id_vars = 'Taxonomy', value_vars = s_list, var_name = 'SampleID', value_name = 'Count')
s = s[s['Count']>0].reset_index(drop=True)
t = s['Taxonomy'].str.split('; ', expand=True)
t = t.rename(columns={0:'Kingdom', 1:'Phylum', 2:'Class', 3:'Order', 4:'Family', 5:'Genus', 6:'Species'})
s = pd.concat([s, t], axis=1)
s = s.replace(['k__', 'p__', 'c__', 'o__', 'f__', 'g__', 's__'], 
              ['','','','','','',''], regex=True)
s = s.replace('', 'Unclassified')
s.to_csv('pmi_taxonomy_control_samples_melted.csv', index=False)
s

,Taxonomy,SampleID,Count,Kingdom,Phylum,Class,Order,Family,Genus,Species
0,Bacteria; Proteobacteria; Alphaproteobacteria;...,10141.NTC.plate1,2.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Bradyrhizobiaceae,Nitrobacter,Unclassified
1,Bacteria; Proteobacteria; Alphaproteobacteria;...,10141.NTC.plate1,2.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Bradyrhizobiaceae,Bradyrhizobium,elkanii
2,Bacteria; Proteobacteria; Alphaproteobacteria;...,10141.NTC.plate1,13.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Bradyrhizobiaceae,Unclassified,Unclassified
3,Bacteria; Proteobacteria; Alphaproteobacteria;...,10141.NTC.plate1,836.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Bradyrhizobiaceae,Unclassified,Unclassified
4,Bacteria; Firmicutes; Bacilli; Bacillales; Pla...,10141.NTC.plate1,2.0,Bacteria,Firmicutes,Bacilli,Bacillales,Planococcaceae,Lysinibacillus,boronitolerans
...,...,...,...,...,...,...,...,...,...,...
10582,Bacteria; Proteobacteria; Gammaproteobacteria;...,714.clean.swab,30.0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Unclassified
10583,Bacteria; Proteobacteria; Alphaproteobacteria;...,714.clean.swab,1.0,Bacteria,Proteobacteria,Alphaproteobacteria,Rhizobiales,Bradyrhizobiaceae,Bradyrhizobium,Unclassified
10584,Bacteria; Proteobacteria; Gammaproteobacteria;...,714.control.MCD2.A2,1.0,Bacteria,Proteobacteria,Gammaproteobacteria,Pseudomonadales,Pseudomonadaceae,Pseudomonas,Unclassified
10585,Bacteria; Proteobacteria; Gammaproteobacteria;...,714.m6ntc.A7,1.0,Bacteria,Proteobacteria,Gammaproteobacteria,Enterobacteriales,Enterobacteriaceae,Unclassified,Unclassified
